In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("database/reporting.db")
log_df = pd.read_sql_query("SELECT * FROM upload_log WHERE table_alias = 'c0_budgetary_execution_details'", conn)
log_df

In [2]:
import json
import os
import logging
from datetime import datetime
from io import BytesIO
import pandas as pd
from great_tables import GT, md, html, google_font, style, loc
from great_tables.data import islands


def insert_variable(report, module, var, value, db_path, anchor=None, gt_table=None):
    try:
        con = sqlite3.connect(db_path)
        logging.debug(f"Inserting variable: report={report}, module={module}, var={var}, anchor={anchor}, db_path={db_path}")
        
        # Serialize the DataFrame data (JSON)
        serialized_value = json.dumps(value, default=str)
        
        # Save GT table as PNG and read as binary
        gt_image = None
        if gt_table:
            # Use a temporary file to save the PNG
            temp_file = f"temp_{var}.png"
            gt_table.save(temp_file)  # Save as PNG (requires playwright)
            with open(temp_file, "rb") as f:
                gt_image = f.read()  # Read as binary
            os.remove(temp_file)  # Clean up
            logging.debug(f"Saved GT table as PNG and read as binary for BLOB storage")
        
        # Insert into database
        con.execute('''
            INSERT INTO report_variables (report_name, module_name, var_name, anchor_name, value, gt_image, created_at)
            VALUES (?, ?, ?, ?, ?, ?, CURRENT_TIMESTAMP)
        ''', (report, module, var, anchor or var, serialized_value, gt_image))
        con.commit()
        logging.debug(f"Successfully inserted variable: {var}")
    except Exception as e:
        logging.error(f"Failed to insert variable {var}: {str(e)}")
        raise
    finally:
        con.close()


def build_commitment_summary_table(df: pd.DataFrame, current_year: int, report: str, db_path: str) -> pd.DataFrame:
    # Filter data
    df = df[df["Budget Period"] == current_year]
    df = df[df["Fund Source"].isin(["VOBU", "EFTA"])]
    df["Programme"] = df["Functional Area Desc"].replace({
        "HORIZONEU_21_27": "HE",
        "H2020_14_20": "H2020"
    })

    # Aggregate data
    agg = df.groupby("Programme")[
        ["Commitment Appropriation", "Committed Amount", "Commitment Available "]
    ].sum().reset_index()
    agg["%"] = agg["Committed Amount"] / agg["Commitment Appropriation"]
    agg = agg.rename(columns={
        "Commitment Appropriation": "Available Commitment Appropriations (1)",
        "Committed Amount": "L1 Commitment (2)",
        "Commitment Available ": "RAL on Appropriation (3)=(1)-(2)",
        "%": "% consumed of L1 and L2 against Commitment Appropriations (4) = (2)/(1)"
    })

    # Filter for HE programme
    agg = agg.loc[agg["Programme"] == "HE"]

    # Create GreatTables object
    table = (
        GT(agg)
        .tab_header(
            title="Commitment Appropriations",
            subtitle="General Overview"
        )
        .tab_style(
            style=style.text(font=google_font(name="IBM Plex Mono")),
            locations=loc.body()
        )
        .tab_stub(rowname_col="Programme")
        .tab_source_note(source_note="Source: Summa DataWharehouse")
        .tab_source_note(
            source_note=md("Table_1a")
        )
        .tab_stubhead(label="Programme")
        .fmt_number(columns=[
            "Available Commitment Appropriations (1)",
            "L1 Commitment (2)",
            "RAL on Appropriation (3)=(1)-(2)"
        ], accounting=True)
        .fmt_percent(columns=[
            "% consumed of L1 and L2 against Commitment Appropriations (4) = (2)/(1)"
        ])
    )

    # Insert data and GT table image into database
    insert_variable(
        report=report,
        module="BudgetModule",
        var="table_1a_commitment_summary",
        value=agg.to_dict(orient="records"),
        db_path=db_path,
        anchor="table_1a",
        gt_table=table
    )

    return agg

ModuleNotFoundError: No module named 'pandas'